In [36]:
# Import required libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Load the data
df = pd.read_csv('convenience_store.csv')  # Make sure to have the CSV in the same directory

# Data Preparation
total_sales = df['store_sales(in millions)'].sum()
total_cost = df['store_cost(in millions)'].sum()
total_profit = total_sales -total_cost
total_units_sold = df['unit_sales(in millions)'].sum()
sales_by_category = df.groupby('food_category')['store_sales(in millions)'].sum().sort_values(ascending=False)
sales_by_department = df.groupby('food_department')['store_sales(in millions)'].sum()
cost_by_department = df.groupby('food_department')['store_cost(in millions)'].sum()
monthly_sales = pd.Series([total_sales * i * 0.01 for i in range(1, 13)], index=[f'Month {i}' for i in range(1, 13)])
top_promotions = df.groupby('promotion_name')['store_sales(in millions)'].sum().sort_values(ascending=False).head(10)

# Create the dashboard layout
fig = make_subplots(
    rows=4, cols=6,
    subplot_titles=("Total Sales", "Total Cost", "Profit/Loss", 
                    "Sales by Food Category",
                    "Sales vs Cost by Food Department", "Monthly Sales Trend", 
                    "Top 10 Promotions by Sales"),
    specs=[[{"type": "indicator", "colspan": 2}, None, {"type": "indicator", "colspan": 2}, None, {"type": "indicator", "colspan": 2}, None],
           [{"type": "bar", "colspan": 6}, None, None, None, None, None],
           [{"type": "scatter", "colspan": 3}, None, None, {"type": "scatter", "colspan": 3}, None, None],
           [{"type": "bar", "colspan": 6}, None, None, None, None, None]],
)



# Add plots
# Add the trace for Total Sales
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_sales,
        number={"prefix": "$", "valueformat": ".2f", "font": {"color": "green"}}
    ),
    row=1, col=1
)

# Add the trace for Total Cost
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_cost,
        number={"prefix": "$", "valueformat": ".2f", "font": {"color": "red"}}
    ),
    row=1, col=3
)

# Add the trace for Total Units Sold
fig.add_trace(
    go.Indicator(
        mode = "delta",
        delta = {'reference': 0},
        value = total_profit,
        number = {"prefix": "$", "valueformat": ".2f"}
    ),
    row=1, col=5
)



## Sales by Food Category
fig.add_trace(
    go.Bar(x=sales_by_category.index, y=sales_by_category.values, name='Sales by Category'),
    row=2, col=1
)

## Sales vs Cost by Food Department
fig.add_trace(
    go.Scatter(x=sales_by_department.index, y=sales_by_department.values, mode='markers', name='Sales'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=cost_by_department.index, y=cost_by_department.values, mode='markers', name='Cost'),
    row=3, col=1
)

## Monthly Sales Trend
fig.add_trace(
    go.Scatter(x=monthly_sales.index, y=monthly_sales.values, mode='lines+markers', name='Monthly Sales'),
    row=3, col=4
)

## Top 10 Promotions by Sales
fig.add_trace(
    go.Bar(x=top_promotions.index, y=top_promotions.values, name='Top Promotions'),
    row=4, col=1
)

# Update layout
fig.update_layout(
    title="Sales Dashboard",
    height=1200,  # You can adjust this value
    width=2400,   # You can adjust this value
    showlegend=False
)

fig.update_xaxes(tickangle=45)


# Create the Dash layout
app.layout = html.Div([
    html.H1("Sales Dashboard"),
    dcc.Graph(id='sales_dashboard', figure=fig)
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [37]:
df

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,...,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4.0,Bag Stuffers,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3.0,Cash Register Lottery,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",59.86
2,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2.0,High Roller Savings,USA,S,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",84.16
3,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2.0,Cash Register Lottery,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,In-Store Coupon,95.78
4,Breakfast Foods,Frozen Foods,Food,4.08,1.4280,3.0,Double Down Sale,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,Radio,50.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60423,Specialty,Carousel,Non-Consumable,2.76,1.3248,1.0,You Save Days,USA,M,F,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,In-Store Coupon,95.25
60424,Specialty,Carousel,Non-Consumable,1.60,0.4960,1.0,Price Cutters,USA,S,F,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,Sunday Paper,69.42
60425,Specialty,Carousel,Non-Consumable,5.52,2.5392,2.0,Weekend Markdown,USA,M,M,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,"Sunday Paper, Radio, TV",67.51
60426,Specialty,Carousel,Non-Consumable,8.28,2.5668,3.0,Sales Days,Canada,S,M,...,27463.0,4193.0,2795.0,1.0,0.0,0.0,0.0,1.0,Sunday Paper,132.88
